# CoOp Training on Google Colab

This notebook implements **Context Optimization (CoOp)** for Vision-Language Models on Google Colab.

## Project Overview
- **Base Framework**: [Dassl.pytorch](https://github.com/KaiyangZhou/Dassl.pytorch)
- **Model**: [CoOp](https://github.com/KaiyangZhou/CoOp) - Prompt Learning for CLIP
- **Dataset**: Oxford-IIIT Pets (37 classes)
- **Experiments**: 1-shot, 4-shot, and 16-shot learning

## Requirements
- GPU: Tesla T4 or better
- Runtime: Python 3.10+
- CUDA: 11.0+

---


In [ ]:
# Clone Dassl.pytorch repository (domain adaptation & semi-supervised learning framework)
!git clone https://github.com/KaiyangZhou/Dassl.pytorch.git

fatal: destination path 'Dassl.pytorch' already exists and is not an empty directory.


In [ ]:
# Change directory to Dassl.pytorch
%cd Dassl.pytorch/

/content/Dassl.pytorch


In [ ]:
# Verify PyTorch and CUDA availability
import torch

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"CUDA version: {torch.version.cuda if torch.cuda.is_available() else 'N/A'}")

PyTorch version: 2.8.0+cu126
CUDA available: True
CUDA version: 12.6


In [ ]:
# Install Dassl dependencies from requirements.txt
# Note: Colab will prompt you to restart runtime after this step
!pip install -r requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.9/69.9 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 185.3/185.3 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 109.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.2/51.2 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 6.4 MB/s eta 0:00:00
  Attempting uninstall: entrypoints
    Found existing installation: entrypoints 0.4
    Uninstalling entrypoints-0.4:
      Successfully uninstalled entrypoints-0.4


## ⚠️ STOP: Restart Runtime Required

After running the cell above, Colab will show a **"RESTART RUNTIME"** button.

### Steps to Continue:
1. Click the **"RESTART RUNTIME"** button (appears above)
2. After restart, **re-run cells 1, 2, and 3** to restore the environment
3. Then continue from **Cell 5** below

**Note**: Do NOT re-run Cell 4 after restart.

---

In [ ]:
# Install Dassl in editable mode for development
# Run this AFTER restarting runtime and re-running cells 1-3
!python setup.py develop

running develop
/usr/local/lib/python3.12/dist-packages/setuptools/command/develop.py:41: EasyInstallDeprecationWarning: easy_install command is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` and ``easy_install``.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        See https://github.com/pypa/setuptools/issues/917 for details.
        ********************************************************************************

!!
  easy_install.initialize_options(self)
/usr/local/lib/python3.12/dist-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        See https://blog

In [ ]:
# Verify Dassl installation after runtime restart
import dassl

print(f"Dassl version: {dassl.__version__}")
print("Dassl installed successfully!")

Dassl version: 0.6.3
Dassl installed successfully!


In [ ]:
# Navigate back to /content directory
%cd /content

/content


In [ ]:
# Clone CoOp (Context Optimization) repository
!git clone https://github.com/KaiyangZhou/CoOp.git

Cloning into 'CoOp'...
remote: Enumerating objects: 455, done.
remote: Counting objects: 100% (250/250), done.
remote: Compressing objects: 100% (51/51), done.
remote: Total 455 (delta 217), reused 199 (delta 199), pack-reused 205 (from 1)
Receiving objects: 100% (455/455), 1.40 MiB | 4.67 MiB/s, done.
Resolving deltas: 100% (266/266), done.


In [ ]:
# Change directory to CoOp
%cd CoOp/

/content/CoOp


In [ ]:
# Install OpenAI CLIP and required packages (ftfy, regex, tqdm)
!pip install ftfy regex tqdm
!pip install git+https://github.com/openai/CLIP.git

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-y0b4kjkb
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-y0b4kjkb
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369490 sha256=9b034ebcd26be722e7b6bd9fbbf54734278c919435d7c82276ea9589a13033e1
  Stored in directory: /tmp/pip-ephem-wheel-cache-yt_rqaeq/wheels/35/3e/df/3d24cbfb3b6a06f17a2bfd7d1138900d4365d9028aa8f6e92f
Successfully built clip


In [ ]:
# Verify CLIP installation and load a test model
import clip
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("RN50", device=device)

print("✓ CLIP loaded successfully!")
print(f"Device: {device}")
print(f"Model: ResNet-50")

100%|███████████████████████████████████████| 256M/256M [00:05<00:00, 45.5MiB/s]


✓ CLIP loaded successfully!
Device: cuda
Model: ResNet-50


## Dataset Setup: Oxford-IIIT Pets

We'll download and prepare the **Oxford-IIIT Pets** dataset:
- **Classes**: 37 pet breeds (dogs and cats)
- **Images**: ~7,400 images
- **Size**: ~800 MB

### Dataset Components:
1. Images (pets photos)
2. Annotations (segmentation masks)
3. Split file (train/val/test division)

---


In [ ]:
# Create main data directory for all datasets
!mkdir -p data/

In [ ]:
# Navigate to data directory
%cd /content/CoOp/data

/content/CoOp/data


In [ ]:
# Create directory for Oxford-IIIT Pets dataset
!mkdir -p oxford_pets
%cd oxford_pets

/content/CoOp/data/oxford_pets


In [ ]:
# Download Oxford Pets images (~800 MB, may take 2-3 minutes)
!wget https://www.robots.ox.ac.uk/~vgg/data/pets/data/images.tar.gz

--2025-11-15 19:21:54--  https://www.robots.ox.ac.uk/~vgg/data/pets/data/images.tar.gz
Resolving www.robots.ox.ac.uk (www.robots.ox.ac.uk)... 129.67.94.2
Connecting to www.robots.ox.ac.uk (www.robots.ox.ac.uk)|129.67.94.2|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://thor.robots.ox.ac.uk/pets/images.tar.gz [following]
--2025-11-15 19:21:55--  https://thor.robots.ox.ac.uk/pets/images.tar.gz
Resolving thor.robots.ox.ac.uk (thor.robots.ox.ac.uk)... 129.67.95.98
Connecting to thor.robots.ox.ac.uk (thor.robots.ox.ac.uk)|129.67.95.98|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 791918971 (755M) [application/octet-stream]
Saving to: ‘images.tar.gz’

images.tar.gz       100%[===================>] 755.23M  22.9MB/s    in 32s     

2025-11-15 19:22:27 (23.5 MB/s) - ‘images.tar.gz’ saved [791918971/791918971]



In [ ]:
# Extract images from compressed archive
!tar -xzf images.tar.gz
!ls -lh

total 756M
drwxr-xr-x 2 1001 1001 304K Jun 18  2012 images
-rw-r--r-- 1 root root 756M Jul 29  2022 images.tar.gz


In [ ]:
# Download annotations (segmentation masks and labels)
!wget https://www.robots.ox.ac.uk/~vgg/data/pets/data/annotations.tar.gz

--2025-11-15 19:22:48--  https://www.robots.ox.ac.uk/~vgg/data/pets/data/annotations.tar.gz
Resolving www.robots.ox.ac.uk (www.robots.ox.ac.uk)... 129.67.94.2
Connecting to www.robots.ox.ac.uk (www.robots.ox.ac.uk)|129.67.94.2|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://thor.robots.ox.ac.uk/pets/annotations.tar.gz [following]
--2025-11-15 19:22:48--  https://thor.robots.ox.ac.uk/pets/annotations.tar.gz
Resolving thor.robots.ox.ac.uk (thor.robots.ox.ac.uk)... 129.67.95.98
Connecting to thor.robots.ox.ac.uk (thor.robots.ox.ac.uk)|129.67.95.98|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19173078 (18M) [application/octet-stream]
Saving to: ‘annotations.tar.gz’

annotations.tar.gz  100%[===================>]  18.28M  11.9MB/s    in 1.5s    

2025-11-15 19:22:51 (11.9 MB/s) - ‘annotations.tar.gz’ saved [19173078/19173078]



In [ ]:
# Extract annotations
!tar -xzf annotations.tar.gz
!ls -lh

total 774M
drwxr-xr-x 4  501 staff 4.0K Jun 30  2012 annotations
-rw-r--r-- 1 root root   19M Jul 29  2022 annotations.tar.gz
drwxr-xr-x 2 1001  1001 304K Jun 18  2012 images
-rw-r--r-- 1 root root  756M Jul 29  2022 images.tar.gz


In [ ]:
# Install gdown if not already installed
!pip install gdown -q

# Download train/val/test split file from Google Drive
!gdown 1501r8Ber4nNKvmlFVQZ8SeUHTcdTTEqs

Downloading...
From: https://drive.google.com/uc?id=1501r8Ber4nNKvmlFVQZ8SeUHTcdTTEqs
To: /content/CoOp/data/oxford_pets/split_zhou_OxfordPets.json
100% 724k/724k [00:00<00:00, 7.73MB/s]


In [ ]:
# Verify dataset structure and files
!echo "=== Main directory ==="
!ls -lh

!echo "\n=== Images directory (first 10) ==="
!ls images/ | head -10

!echo "\n=== Annotations directory ==="
!ls annotations/

=== Main directory ===
total 775M
drwxr-xr-x 4  501 staff 4.0K Jun 30  2012 annotations
-rw-r--r-- 1 root root   19M Jul 29  2022 annotations.tar.gz
drwxr-xr-x 2 1001  1001 304K Jun 18  2012 images
-rw-r--r-- 1 root root  756M Jul 29  2022 images.tar.gz
-rw-r--r-- 1 root root  708K Sep  5  2021 split_zhou_OxfordPets.json
\n=== Images directory (first 10) ===
Abyssinian_100.jpg
Abyssinian_100.mat
Abyssinian_101.jpg
Abyssinian_101.mat
Abyssinian_102.jpg
Abyssinian_102.mat
Abyssinian_103.jpg
Abyssinian_104.jpg
Abyssinian_105.jpg
Abyssinian_106.jpg
\n=== Annotations directory ===
list.txt  README  test.txt  trainval.txt  trimaps  xmls


In [ ]:
# Remove compressed files to free up disk space (~775 MB)
!rm images.tar.gz annotations.tar.gz
!echo "✓ Compressed files removed successfully"

✓ Compressed files removed successfully


## Bug Fix: PyTorch 2.x Compatibility

CoOp and Dassl were developed for older PyTorch versions. We need to fix compatibility issues with PyTorch 2.x:

### Issue 1: LRScheduler `verbose` Parameter
- **Problem**: PyTorch 2.0+ removed the `verbose` parameter from `LRScheduler.__init__()`
- **Solution**: Remove `verbose` parameter from Dassl's lr_scheduler.py

This fix is required for training to work properly.

---


In [ ]:
# Navigate to Dassl's optimizer module directory
%cd /content/Dassl.pytorch/dassl/optim/

/content/Dassl.pytorch/dassl/optim


In [ ]:
# Display the problematic lines in lr_scheduler.py
!sed -n '15,50p' lr_scheduler.py

        successor,
        warmup_epoch,
        last_epoch=-1,
        verbose=False
    ):
        self.successor = successor
        self.warmup_epoch = warmup_epoch
        super().__init__(optimizer, last_epoch, verbose)

    def get_lr(self):
        raise NotImplementedError

    def step(self, epoch=None):
        if self.last_epoch >= self.warmup_epoch:
            self.successor.step(epoch)
            self._last_lr = self.successor.get_last_lr()
        else:
            super().step(epoch)


class ConstantWarmupScheduler(_BaseWarmupScheduler):

    def __init__(
        self,
        optimizer,
        successor,
        warmup_epoch,
        cons_lr,
        last_epoch=-1,
        verbose=False
    ):
        self.cons_lr = cons_lr
        super().__init__(
            optimizer, successor, warmup_epoch, last_epoch, verbose
        )



In [ ]:
# Fix: Remove 'verbose' parameter from LRScheduler.__init__() calls
# This fixes compatibility with PyTorch 2.0+
!sed -i 's/super().__init__(optimizer, last_epoch, verbose)/super().__init__(optimizer, last_epoch)/g' lr_scheduler.py

!echo "✓ LRScheduler compatibility fix applied"

✓ LRScheduler compatibility fix applied


In [ ]:
# Verify the fix was applied correctly
!grep -n "super().__init__" lr_scheduler.py | head -10

22:        super().__init__(optimizer, last_epoch)
47:        super().__init__(
69:        super().__init__(


In [ ]:
# Navigate back to CoOp root directory
%cd /content/CoOp
!pwd

/content/CoOp
/content/CoOp


In [ ]:
# Display dataset configuration (classes, transforms, paths)
!echo "=== Oxford Pets Dataset Configuration ==="
!cat configs/datasets/oxford_pets.yaml

!echo "\n=== Trainer Configuration ==="
!cat configs/trainers/CoOp/vit_b16_ep50.yaml | head -30

=== Oxford Pets Dataset Configuration ===
DATASET:
  NAME: "OxfordPets"\n=== Trainer Configuration ===
DATALOADER:
  TRAIN_X:
    BATCH_SIZE: 32
  TEST:
    BATCH_SIZE: 100
  NUM_WORKERS: 8

INPUT:
  SIZE: (224, 224)
  INTERPOLATION: "bicubic"
  PIXEL_MEAN: [0.48145466, 0.4578275, 0.40821073]
  PIXEL_STD: [0.26862954, 0.26130258, 0.27577711]
  TRANSFORMS: ["random_resized_crop", "random_flip", "normalize"]

OPTIM:
  NAME: "sgd"
  LR: 0.002
  MAX_EPOCH: 50
  LR_SCHEDULER: "cosine"
  WARMUP_EPOCH: 1
  WARMUP_TYPE: "constant"
  WARMUP_CONS_LR: 1e-5

TRAIN:
  PRINT_FREQ: 5

MODEL:
  BACKBONE:
    NAME: "ViT-B/16"

## Training Experiments

We'll train CoOp with three different configurations:
1. **1-shot**: 1 training sample per class (quick test, ~5 min)
2. **4-shot**: 4 training samples per class (~15 min)
3. **16-shot**: 16 training samples per class (~30 min)

### Training Configuration:
- **Backbone**: ViT-B/16 (Vision Transformer)
- **Epochs**: 50
- **Context Tokens**: 16 learnable tokens
- **Optimizer**: SGD with cosine learning rate schedule

---


In [ ]:
# Training: 1-shot learning (quick test, ~5 minutes)
# Purpose: Verify that training pipeline works correctly
!python train.py \
--root /content/CoOp/data \
--seed 1 \
--trainer CoOp \
--dataset-config-file configs/datasets/oxford_pets.yaml \
--config-file configs/trainers/CoOp/vit_b16_ep50.yaml \
--output-dir output/oxford_pets_1shot \
DATASET.NUM_SHOTS 1

2025-11-15 19:23:42.551184: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763234622.571974    1768 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763234622.578296    1768 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1763234622.593772    1768 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1763234622.593798    1768 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1763234622.593802    1768 computation_placer.cc:177] computation placer alr

In [ ]:
# Display training results for 1-shot experiment
!echo "=== Training Log (last 30 lines) ==="
!tail -30 output/oxford_pets_1shot/log.txt

!echo "\n=== Final Accuracy ==="
!grep "accuracy:" output/oxford_pets_1shot/log.txt | tail -3

=== Training Log (last 30 lines) ===
epoch [32/50] batch [1/1] time 0.543 (0.543) data 0.398 (0.398) loss 0.3357 (0.3357) acc 90.6250 (90.6250) lr 6.3188e-04 eta 0:00:09
epoch [33/50] batch [1/1] time 0.533 (0.533) data 0.388 (0.388) loss 0.5142 (0.5142) acc 87.5000 (87.5000) lr 5.7422e-04 eta 0:00:09
epoch [34/50] batch [1/1] time 0.538 (0.538) data 0.387 (0.387) loss 0.4978 (0.4978) acc 87.5000 (87.5000) lr 5.1825e-04 eta 0:00:08
epoch [35/50] batch [1/1] time 0.544 (0.544) data 0.397 (0.397) loss 0.3496 (0.3496) acc 93.7500 (93.7500) lr 4.6417e-04 eta 0:00:08
epoch [36/50] batch [1/1] time 0.528 (0.528) data 0.383 (0.383) loss 0.5059 (0.5059) acc 87.5000 (87.5000) lr 4.1221e-04 eta 0:00:07
epoch [37/50] batch [1/1] time 0.535 (0.535) data 0.395 (0.395) loss 0.4453 (0.4453) acc 93.7500 (93.7500) lr 3.6258e-04 eta 0:00:06
epoch [38/50] batch [1/1] time 0.561 (0.561) data 0.418 (0.418) loss 0.3726 (0.3726) acc 90.6250 (90.6250) lr 3.1545e-04 eta 0:00:06
epoch [39/50] batch [1/1] time 0

In [ ]:
# Training: 4-shot learning (~15 minutes)
# Expected accuracy: ~92-93%
!python train.py \
--root /content/CoOp/data \
--seed 1 \
--trainer CoOp \
--dataset-config-file configs/datasets/oxford_pets.yaml \
--config-file configs/trainers/CoOp/vit_b16_ep50.yaml \
--output-dir output/oxford_pets_4shot \
DATASET.NUM_SHOTS 4

2025-11-15 19:25:56.508059: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763234756.530716    3434 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763234756.536992    3434 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1763234756.553766    3434 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1763234756.553798    3434 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1763234756.553802    3434 computation_placer.cc:177] computation placer alr

In [ ]:
# Display training results for 4-shot experiment
!echo "=== Training Log (last 30 lines) ==="
!tail -30 output/oxford_pets_4shot/log.txt

!echo "\n=== Final Accuracy ==="
!grep "accuracy:" output/oxford_pets_4shot/log.txt | tail -3

=== Training Log (last 30 lines) ===
epoch [46/50] batch [2/4] time 0.155 (0.580) data 0.000 (0.427) loss 0.3196 (0.3351) acc 90.6250 (90.6250) lr 7.0224e-05 eta 0:00:10
epoch [46/50] batch [3/4] time 0.154 (0.438) data 0.000 (0.285) loss 0.1918 (0.2873) acc 96.8750 (92.7083) lr 7.0224e-05 eta 0:00:07
epoch [46/50] batch [4/4] time 0.155 (0.368) data 0.000 (0.214) loss 0.2341 (0.2740) acc 96.8750 (93.7500) lr 4.8943e-05 eta 0:00:05
epoch [47/50] batch [1/4] time 0.777 (0.777) data 0.593 (0.593) loss 0.2712 (0.2712) acc 93.7500 (93.7500) lr 4.8943e-05 eta 0:00:11
epoch [47/50] batch [2/4] time 0.276 (0.527) data 0.125 (0.359) loss 0.5132 (0.3922) acc 87.5000 (90.6250) lr 4.8943e-05 eta 0:00:07
epoch [47/50] batch [3/4] time 0.162 (0.405) data 0.000 (0.239) loss 0.3152 (0.3665) acc 96.8750 (92.7083) lr 4.8943e-05 eta 0:00:05
epoch [47/50] batch [4/4] time 0.155 (0.342) data 0.000 (0.180) loss 0.5000 (0.3999) acc 90.6250 (92.1875) lr 3.1417e-05 eta 0:00:04
epoch [48/50] batch [1/4] time 0

In [ ]:
# Training: 16-shot learning (~30 minutes)
# This is the standard benchmark configuration
# Expected accuracy: ~93-94%
!python train.py \
--root /content/CoOp/data \
--seed 1 \
--trainer CoOp \
--dataset-config-file configs/datasets/oxford_pets.yaml \
--config-file configs/trainers/CoOp/vit_b16_ep50.yaml \
--output-dir output/oxford_pets_16shot \
DATASET.NUM_SHOTS 16

2025-11-15 19:28:45.861756: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763234925.881885    5568 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763234925.888364    5568 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1763234925.904109    5568 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1763234925.904137    5568 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1763234925.904140    5568 computation_placer.cc:177] computation placer alr

In [ ]:
# Display training results for 16-shot experiment
!echo "=== Training Log (last 30 lines) ==="
!tail -30 output/oxford_pets_16shot/log.txt

!echo "\n=== Final Accuracy ==="
!grep "accuracy:" output/oxford_pets_16shot/log.txt | tail -3

=== Training Log (last 30 lines) ===
epoch [44/50] batch [15/18] time 0.153 (0.303) data 0.000 (0.114) loss 0.4646 (0.5366) acc 90.6250 (82.9167) lr 1.2369e-04 eta 0:00:33
epoch [45/50] batch [5/18] time 0.164 (0.691) data 0.002 (0.419) loss 0.3828 (0.5525) acc 87.5000 (83.7500) lr 9.5173e-05 eta 0:01:11
epoch [45/50] batch [10/18] time 0.160 (0.423) data 0.000 (0.210) loss 0.5649 (0.4924) acc 78.1250 (85.6250) lr 9.5173e-05 eta 0:00:41
epoch [45/50] batch [15/18] time 0.154 (0.333) data 0.000 (0.140) loss 0.3752 (0.4711) acc 84.3750 (86.2500) lr 9.5173e-05 eta 0:00:30
epoch [46/50] batch [5/18] time 0.321 (0.596) data 0.002 (0.268) loss 0.5186 (0.5530) acc 78.1250 (83.1250) lr 7.0224e-05 eta 0:00:50
epoch [46/50] batch [10/18] time 0.154 (0.376) data 0.000 (0.134) loss 0.3618 (0.4944) acc 90.6250 (86.2500) lr 7.0224e-05 eta 0:00:30
epoch [46/50] batch [15/18] time 0.153 (0.303) data 0.000 (0.090) loss 0.2493 (0.4785) acc 90.6250 (85.8333) lr 7.0224e-05 eta 0:00:22
epoch [47/50] batch 

## Results Comparison

Let's compare the performance across all three experiments to observe how accuracy improves with more training samples per class.

### Expected Trends:
- **1-shot**: ~90-91% (limited data)
- **4-shot**: ~92-93% (moderate improvement)
- **16-shot**: ~93-94% (best performance)

---


In [ ]:
# Compare accuracy across all experiments
!echo "═══════════════════════════════════════════"
!echo "           RESULTS COMPARISON"
!echo "═══════════════════════════════════════════"

!echo "\n📊 1-Shot Results:"
!grep "=> result" -A 6 output/oxford_pets_1shot/log.txt | tail -6

!echo "\n📊 4-Shot Results:"
!grep "=> result" -A 6 output/oxford_pets_4shot/log.txt | tail -6

!echo "\n📊 16-Shot Results:"
!grep "=> result" -A 6 output/oxford_pets_16shot/log.txt | tail -6


═══════════════════════════════════════════
           RESULTS COMPARISON
═══════════════════════════════════════════
\n📊 1-Shot Results:
* total: 3,669
* correct: 3,328
* accuracy: 90.7%
* error: 9.3%
* macro_f1: 90.6%
Elapsed: 0:01:06
\n📊 4-Shot Results:
* total: 3,669
* correct: 3,398
* accuracy: 92.6%
* error: 7.4%
* macro_f1: 92.5%
Elapsed: 0:01:54
\n📊 16-Shot Results:
* total: 3,669
* correct: 3,426
* accuracy: 93.4%
* error: 6.6%
* macro_f1: 93.3%
Elapsed: 0:05:12


In [ ]:
# Display saved model checkpoints for 16-shot experiment
!echo "=== Saved Checkpoints ==="
!ls -lh output/oxford_pets_16shot/

!echo "\n=== Prompt Learner Models ==="
!ls -lh output/oxford_pets_16shot/prompt_learner/

=== Saved Checkpoints ===
total 44K
-rw-r--r-- 1 root root  30K Nov 15 19:34 log.txt
drwxr-xr-x 2 root root 4.0K Nov 15 19:33 prompt_learner
drwxr-xr-x 2 root root 4.0K Nov 15 19:29 tensorboard
\n=== Prompt Learner Models ===
total 2.3M
-rw-r--r-- 1 root root   17 Nov 15 19:33 checkpoint
-rw-r--r-- 1 root root 2.3M Nov 15 19:33 model.pth.tar-50


In [ ]:
# Display the actual training configuration used (from log file)
!echo "=== Training Configuration (16-shot) ==="
!sed -n '/\*\* Config \*\*/,/Loading trainer/p' output/oxford_pets_16shot/log.txt | head -60

=== Training Configuration (16-shot) ===
** Config **
************
DATALOADER:
  K_TRANSFORMS: 1
  NUM_WORKERS: 8
  RETURN_IMG0: False
  TEST:
    BATCH_SIZE: 100
    SAMPLER: SequentialSampler
  TRAIN_U:
    BATCH_SIZE: 32
    N_DOMAIN: 0
    N_INS: 16
    SAME_AS_X: True
    SAMPLER: RandomSampler
  TRAIN_X:
    BATCH_SIZE: 32
    N_DOMAIN: 0
    N_INS: 16
    SAMPLER: RandomSampler
DATASET:
  ALL_AS_UNLABELED: False
  CIFAR_C_LEVEL: 1
  CIFAR_C_TYPE: 
  NAME: OxfordPets
  NUM_LABELED: -1
  NUM_SHOTS: 16
  ROOT: /content/CoOp/data
  SOURCE_DOMAINS: ()
  STL10_FOLD: -1
  SUBSAMPLE_CLASSES: all
  TARGET_DOMAINS: ()
  VAL_PERCENT: 0.1
INPUT:
  COLORJITTER_B: 0.4
  COLORJITTER_C: 0.4
  COLORJITTER_H: 0.1
  COLORJITTER_S: 0.4
  CROP_PADDING: 4
  CUTOUT_LEN: 16
  CUTOUT_N: 1
  GB_K: 21
  GB_P: 0.5
  GN_MEAN: 0.0
  GN_STD: 0.15
  INTERPOLATION: bicubic
  NO_TRANSFORM: False
  PIXEL_MEAN: [0.48145466, 0.4578275, 0.40821073]
  PIXEL_STD: [0.26862954, 0.26130258, 0.27577711]
  RANDAUGMENT_M: 1

## Save Results to Google Drive

To preserve your trained models and results permanently, we'll save them to Google Drive.

### What Will Be Saved:
- Trained model checkpoints (`.pth.tar` files)
- Training logs (`.txt` files)
- Configuration files
- All experiment outputs

**Note**: Make sure you have enough space in Google Drive (~200-300 MB per experiment).

---


In [ ]:
# Mount Google Drive to save results permanently
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Copy all experiment outputs to Google Drive
!mkdir -p /content/drive/MyDrive/CoOp_OxfordPets_Results

!cp -r /content/CoOp/output/* /content/drive/MyDrive/CoOp_OxfordPets_Results/

!echo "✓ All results saved to Google Drive!"
!echo "📁 Location: MyDrive/CoOp_OxfordPets_Results/"

✓ All results saved to Google Drive!
📁 Location: MyDrive/CoOp_OxfordPets_Results/


In [ ]:
# Create a compressed archive of all results for easy download
!cd /content/CoOp && zip -r oxford_pets_results.zip output/

!echo "✓ Results compressed successfully!"
!ls -lh oxford_pets_results.zip

  adding: output/ (stored 0%)
  adding: output/oxford_pets_16shot/ (stored 0%)
  adding: output/oxford_pets_16shot/log.txt (deflated 73%)
  adding: output/oxford_pets_16shot/tensorboard/ (stored 0%)
  adding: output/oxford_pets_16shot/tensorboard/events.out.tfevents.1763234940.d3c8ed2ed7b1.5568.0 (deflated 71%)
  adding: output/oxford_pets_16shot/prompt_learner/ (stored 0%)
  adding: output/oxford_pets_16shot/prompt_learner/model.pth.tar-50 (deflated 92%)
  adding: output/oxford_pets_16shot/prompt_learner/checkpoint (stored 0%)
  adding: output/oxford_pets_1shot/ (stored 0%)
  adding: output/oxford_pets_1shot/log.txt (deflated 66%)
  adding: output/oxford_pets_1shot/tensorboard/ (stored 0%)
  adding: output/oxford_pets_1shot/tensorboard/events.out.tfevents.1763234644.d3c8ed2ed7b1.1768.0 (deflated 67%)
  adding: output/oxford_pets_1shot/prompt_learner/ (stored 0%)
  adding: output/oxford_pets_1shot/prompt_learner/model.pth.tar-50 (deflated 92%)
  adding: output/oxford_pets_1shot/prompt_

In [ ]:
# Generate summary statistics table
import pandas as pd

# Extract accuracies from log files
results = {
    'Experiment': ['1-Shot', '4-Shot', '16-Shot'],
    'Training Samples': [37, 148, 592],
    'Accuracy (%)': [],
    'Macro F1 (%)': []
}

# Parse results (simplified - you can enhance this)
!echo "=== EXPERIMENT SUMMARY ==="
!echo "┌─────────────┬──────────────────┬──────────────┬─────────────┐"
!echo "│ Experiment  │ Training Samples │ Accuracy (%) │ Macro F1 (%)│"
!echo "├─────────────┼──────────────────┼──────────────┼─────────────┤"
!echo "│ 1-Shot      │       37         │    ~90.7     │    ~90.6    │"
!echo "│ 4-Shot      │      148         │    ~92.6     │    ~92.5    │"
!echo "│ 16-Shot     │      592         │    ~93.0+    │    ~93.0+   │"
!echo "└─────────────┴──────────────────┴──────────────┴─────────────┘"


=== EXPERIMENT SUMMARY ===
┌─────────────┬──────────────────┬──────────────┬─────────────┐
│ Experiment  │ Training Samples │ Accuracy (%) │ Macro F1 (%)│
├─────────────┼──────────────────┼──────────────┼─────────────┤
│ 1-Shot      │       37         │    ~90.7     │    ~90.6    │
│ 4-Shot      │      148         │    ~92.6     │    ~92.5    │
│ 16-Shot     │      592         │    ~93.0+    │    ~93.0+   │
└─────────────┴──────────────────┴──────────────┴─────────────┘


## Advanced: Custom Hyperparameter Experiments

Now that we've established baseline results, let's explore how different hyperparameters affect model performance.

### Hyperparameters to Experiment With:
1. **Context Length** (`N_CTX`): Number of learnable prompt tokens
2. **Learning Rate** (`LR`): Training speed
3. **Class-Specific Context** (`CSC`): Separate prompts per class
4. **Backbone Model**: Different CLIP architectures

---


In [ ]:
# Experiment: Train with 8 context tokens instead of 16
# Expected: Slightly lower accuracy but faster training
!python train.py \
--root /content/CoOp/data \
--seed 1 \
--trainer CoOp \
--dataset-config-file configs/datasets/oxford_pets.yaml \
--config-file configs/trainers/CoOp/vit_b16_ep50.yaml \
--output-dir output/oxford_pets_custom_ctx8 \
DATASET.NUM_SHOTS 16 \
TRAINER.COOP.N_CTX 8

2025-11-15 19:37:09.288187: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763235429.311641    9743 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763235429.318085    9743 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1763235429.334323    9743 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1763235429.334349    9743 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1763235429.334353    9743 computation_placer.cc:177] computation placer alr

In [ ]:
# Experiment: Enable class-specific context (CSC)
# Each class gets its own learnable prompt
# Expected: Potentially higher accuracy but more parameters
!python train.py \
--root /content/CoOp/data \
--seed 1 \
--trainer CoOp \
--dataset-config-file configs/datasets/oxford_pets.yaml \
--config-file configs/trainers/CoOp/vit_b16_ep50.yaml \
--output-dir output/oxford_pets_custom_csc \
DATASET.NUM_SHOTS 16 \
TRAINER.COOP.CSC True

2025-11-15 19:43:03.730531: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763235783.751785   13056 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763235783.758414   13056 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1763235783.775454   13056 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1763235783.775482   13056 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1763235783.775486   13056 computation_placer.cc:177] computation placer alr

In [ ]:
# Experiment: Train with lower learning rate (0.001 instead of 0.002)
# Expected: Slower convergence but potentially better final accuracy
!python train.py \
--root /content/CoOp/data \
--seed 1 \
--trainer CoOp \
--dataset-config-file configs/datasets/oxford_pets.yaml \
--config-file configs/trainers/CoOp/vit_b16_ep50.yaml \
--output-dir output/oxford_pets_custom_lr \
DATASET.NUM_SHOTS 16 \
OPTIM.LR 0.001

2025-11-15 19:49:34.401970: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763236174.423494   16516 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763236174.430170   16516 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1763236174.446099   16516 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1763236174.446125   16516 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1763236174.446131   16516 computation_placer.cc:177] computation placer alr

In [ ]:
# Compare results of custom experiments
!echo "═══════════════════════════════════════════════════"
!echo "       CUSTOM EXPERIMENTS COMPARISON"
!echo "═══════════════════════════════════════════════════"

!echo "\n🔧 Baseline (16-shot, N_CTX=16, CSC=False, LR=0.002):"
!grep "=> result" -A 3 output/oxford_pets_16shot/log.txt | tail -3

!echo "\n🔧 Custom: N_CTX=8:"
!grep "=> result" -A 3 output/oxford_pets_custom_ctx8/log.txt | tail -3

!echo "\n🔧 Custom: CSC=True:"
!grep "=> result" -A 3 output/oxford_pets_custom_csc/log.txt | tail -3

!echo "\n🔧 Custom: LR=0.001:"
!grep "=> result" -A 3 output/oxford_pets_custom_lr/log.txt | tail -3

═══════════════════════════════════════════════════
       CUSTOM EXPERIMENTS COMPARISON
═══════════════════════════════════════════════════
\n🔧 Baseline (16-shot, N_CTX=16, CSC=False, LR=0.002):
* total: 3,669
* correct: 3,426
* accuracy: 93.4%
\n🔧 Custom: N_CTX=8:
* total: 3,669
* correct: 3,411
* accuracy: 93.0%
\n🔧 Custom: CSC=True:
* total: 3,669
* correct: 3,306
* accuracy: 90.1%
\n🔧 Custom: LR=0.001:
* total: 3,669
* correct: 3,415
* accuracy: 93.1%


## Visualization & Analysis (Optional)

These cells provide additional analysis and visualization of your results. They are optional but recommended for deeper understanding.

---


In [ ]:
# Analyze training time for each experiment
%%bash
echo "═══════════════════════════════════════════"
echo "        TRAINING TIME ANALYSIS"
echo "═══════════════════════════════════════════"

for exp in 1shot 4shot 16shot; do
    echo -e "\n📊 oxford_pets_${exp}:"
    grep "Elapsed:" output/oxford_pets_${exp}/log.txt | tail -1
done


═══════════════════════════════════════════
        TRAINING TIME ANALYSIS
═══════════════════════════════════════════

📊 oxford_pets_1shot:
Elapsed: 0:01:06

📊 oxford_pets_4shot:
Elapsed: 0:01:54

📊 oxford_pets_16shot:
Elapsed: 0:05:12


In [ ]:
# Extract and display learned prompt tokens (advanced)
import torch

# Load 16-shot model checkpoint
checkpoint_path = "output/oxford_pets_16shot/prompt_learner/model.pth.tar-50"

print("=== Learned Prompt Analysis ===")
print(f"Loading checkpoint from: {checkpoint_path}\n")

try:
    # Load with weights_only=False for compatibility
    checkpoint = torch.load(checkpoint_path, map_location='cpu', weights_only=False)

    print("✓ Checkpoint loaded successfully!")
    print(f"Checkpoint keys: {list(checkpoint.keys())}")
    print(f"\nModel state_dict keys: {len(checkpoint['state_dict'])} parameters")

except Exception as e:
    print(f"⚠️ Error loading checkpoint: {e}")

=== Learned Prompt Analysis ===
Loading checkpoint from: output/oxford_pets_16shot/prompt_learner/model.pth.tar-50

✓ Checkpoint loaded successfully!
Checkpoint keys: ['state_dict', 'epoch', 'optimizer', 'scheduler', 'val_result']

Model state_dict keys: 3 parameters


In [ ]:
# Display summary of all experiments and their outputs
!echo "═══════════════════════════════════════════"
!echo "      ALL EXPERIMENTS DIRECTORY"
!echo "═══════════════════════════════════════════"
!ls -lh output/

!echo "\n📊 Total disk space used by experiments:"
!du -sh output/

═══════════════════════════════════════════
      ALL EXPERIMENTS DIRECTORY
═══════════════════════════════════════════
total 24K
drwxr-xr-x 4 root root 4.0K Nov 15 19:33 oxford_pets_16shot
drwxr-xr-x 4 root root 4.0K Nov 15 19:24 oxford_pets_1shot
drwxr-xr-x 4 root root 4.0K Nov 15 19:27 oxford_pets_4shot
drwxr-xr-x 4 root root 4.0K Nov 15 19:48 oxford_pets_custom_csc
drwxr-xr-x 4 root root 4.0K Nov 15 19:42 oxford_pets_custom_ctx8
drwxr-xr-x 4 root root 4.0K Nov 15 19:54 oxford_pets_custom_lr
\n📊 Total disk space used by experiments:
16M	output/


## Project Summary & Next Steps

### 🎯 What We Accomplished:
- ✅ Set up CoOp training environment on Google Colab
- ✅ Fixed PyTorch 2.x compatibility issues
- ✅ Downloaded and prepared Oxford-IIIT Pets dataset
- ✅ Trained models with 1-shot, 4-shot, and 16-shot configurations
- ✅ Achieved ~93% accuracy on 16-shot learning
- ✅ Experimented with custom hyperparameters

### 📚 Next Steps:
1. Try different datasets (e.g., Caltech101, Food101)
2. Experiment with CoCoOp (improved version of CoOp)
3. Test on domain shift scenarios
4. Fine-tune for your specific use case

### 📖 References:
- **Paper**: [Learning to Prompt for Vision-Language Models (IJCV 2022)](https://arxiv.org/abs/2109.01134)
- **CoOp Repo**: [https://github.com/KaiyangZhou/CoOp](https://github.com/KaiyangZhou/CoOp)
- **Dassl Repo**: [https://github.com/KaiyangZhou/Dassl.pytorch](https://github.com/KaiyangZhou/Dassl.pytorch)

---


In [ ]:
# Generate a comprehensive final report
!echo "╔═══════════════════════════════════════════════════════════╗"
!echo "║                   FINAL PROJECT REPORT                    ║"
!echo "╚═══════════════════════════════════════════════════════════╝"

!echo "\n📋 PROJECT INFORMATION:"
!echo "  • Dataset: Oxford-IIIT Pets (37 classes)"
!echo "  • Model: CoOp (Context Optimization for CLIP)"
!echo "  • Backbone: ViT-B/16"
!echo "  • Experiments: 6 training runs"

!echo "\n📊 MAIN RESULTS:"
!echo "  ┌─────────────┬──────────────┬─────────────┐"
!echo "  │ Experiment  │ Accuracy (%) │ Macro F1 (%)│"
!echo "  ├─────────────┼──────────────┼─────────────┤"
!echo "  │ 1-Shot      │    ~90.7     │    ~90.6    │"
!echo "  │ 4-Shot      │    ~92.6     │    ~92.5    │"
!echo "  │ 16-Shot     │    ~93.0+    │    ~93.0+   │"
!echo "  └─────────────┴──────────────┴─────────────┘"

!echo "\n💾 SAVED OUTPUTS:"
!echo "  • Local: /content/CoOp/output/"
!echo "  • Google Drive: MyDrive/CoOp_OxfordPets_Results/"
!echo "  • Archive: oxford_pets_results.zip"

!echo "\n✅ PROJECT STATUS: COMPLETED SUCCESSFULLY"
!echo "\n═══════════════════════════════════════════════════════════"

╔═══════════════════════════════════════════════════════════╗
║                   FINAL PROJECT REPORT                    ║
╚═══════════════════════════════════════════════════════════╝
\n📋 PROJECT INFORMATION:
  • Dataset: Oxford-IIIT Pets (37 classes)
  • Model: CoOp (Context Optimization for CLIP)
  • Backbone: ViT-B/16
  • Experiments: 6 training runs
\n📊 MAIN RESULTS:
  ┌─────────────┬──────────────┬─────────────┐
  │ Experiment  │ Accuracy (%) │ Macro F1 (%)│
  ├─────────────┼──────────────┼─────────────┤
  │ 1-Shot      │    ~90.7     │    ~90.6    │
  │ 4-Shot      │    ~92.6     │    ~92.5    │
  │ 16-Shot     │    ~93.0+    │    ~93.0+   │
  └─────────────┴──────────────┴─────────────┘
\n💾 SAVED OUTPUTS:
  • Local: /content/CoOp/output/
  • Google Drive: MyDrive/CoOp_OxfordPets_Results/
  • Archive: oxford_pets_results.zip
\n✅ PROJECT STATUS: COMPLETED SUCCESSFULLY
\n═══════════════════════════════════════════════════════════


## 🎓 Acknowledgments & License

This notebook is based on the following repositories:

### Original Works:
- **CoOp**: [KaiyangZhou/CoOp](https://github.com/KaiyangZhou/CoOp) (MIT License)
- **Dassl.pytorch**: [KaiyangZhou/Dassl.pytorch](https://github.com/KaiyangZhou/Dassl.pytorch) (MIT License)

### Citation:
If you use this code in your research, please cite:

